# Exploratory Analysis — Clinical Trials CSV

This notebook performs an initial exploration of `../raw/clin_trials.csv` to understand the data structure, missing values, and identify potential categorical columns.
```

In [ ]:
import pandas as pd

df = pd.read_csv('../raw/clin_trials.csv')

# Basic Exploration

Check the shape and column data types to understand the dataset size and structure.
```

In [52]:
print("_" * 60)
print(f"Shape: {df.shape}")
print(f"\nData types:")
print(df.dtypes)

____________________________________________________________
Shape: (496615, 17)

Data types:
Unnamed: 0                   int64
Organization Full Name      object
Organization Class          object
Responsible Party           object
Brief Title                 object
Full Title                  object
Overall Status              object
Start Date                  object
Standard Age                object
Conditions                  object
Primary Purpose             object
Interventions               object
Intervention Description    object
Study Type                  object
Phases                      object
Outcome Measure             object
Medical Subject Headings    object
dtype: object


## Missing Values Analysis

Compute missing value counts and percentages for each column. Only columns with missing values are displayed.
```

In [53]:
print("_" * 60)
print("Missing values")
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing, 
    'missing_percent': missing_percent
})
print(missing_df[missing_df['missing_count'] > 0].sort_values(by='missing_percent', ascending=False))

____________________________________________________________
Missing values
              missing_count  missing_percent
Phases               183844        37.019421
Standard Age            885         0.178206


## Sample Data Preview

Display the first few rows to inspect actual data values.
```

In [54]:
print("_" * 60)
print("Sample data")
display(df.head(10))

____________________________________________________________
Sample data


,Unnamed: 0,Organization Full Name,Organization Class,Responsible Party,Brief Title,Full Title,Overall Status,Start Date,Standard Age,Conditions,Primary Purpose,Interventions,Intervention Description,Study Type,Phases,Outcome Measure,Medical Subject Headings
0,0,Montefiore Medical Center,OTHER,SPONSOR,Kinesiotape for Edema After Bilateral Total Kn...,"Effect of Kinesiotaping on Edema Management, P...",COMPLETED,2021-10-18,ADULT OLDER_ADULT,"Arthroplasty Complications, Arthroplasty, Repl...",TREATMENT,Kinesio(R)Tape for edema control,"Kinesio(R)Tape is an elastic, cotton tape with...",INTERVENTIONAL,NaN,Change from baseline and during 1-2-day time i...,Edema
1,1,Seoul National University Hospital,OTHER,Unknown,An Open-labeled Trial of Ramipril in Patients ...,An Open-labeled Trial of Ramipril in Patients ...,COMPLETED,2004-10,ADULT OLDER_ADULT,Migraine With Hypertension,TREATMENT,Ramipril,ramipril 2.5mg twice a day,INTERVENTIONAL,PHASE2,headache frequency,Migraine Disorders Hypertension
2,2,Federico II University,OTHER,PRINCIPAL_INVESTIGATOR,OCTA in Epivascular Glia After Dex Implant,Evaluation of Changes in Epivascular Glia Befo...,COMPLETED,2021-01-01,ADULT OLDER_ADULT,Diabetic Retinopathy,Unknown,Dexamethasone intravitreal implant,Dexamethasone intravitreal implant,OBSERVATIONAL,Unknown,Changes in epivascular glia after Dexamethason...,Diabetic Retinopathy
3,3,Sidney Kimmel Comprehensive Cancer Center at J...,OTHER,SPONSOR,Preoperative Immune Checkpoint Inhibitor for P...,Preoperative Immune Checkpoint Inhibitor Thera...,COMPLETED,2019-07-08,ADULT OLDER_ADULT,"Head and Neck Squamous Cell Carcinoma, Head an...",TREATMENT,Nivolumab 480mg and surgical resection,One dose of Nivolumab 480mg given four weeks p...,INTERVENTIONAL,PHASE2,Safety as measured by number of participants w...,"Carcinoma Carcinoma, Squamous Cell Head and Ne..."
4,4,Northwestern University,OTHER,SPONSOR,Genistein in Treating Patients With Prostate C...,Phase 2 Trial of Genistein in Men With Circula...,TERMINATED,2011-02-03,ADULT OLDER_ADULT,"Adenocarcinoma of the Prostate, Recurrent Pros...",TREATMENT,"genistein, placebo, therapeutic conventional s...","Given orally, Given orally, Radical prostatect...",INTERVENTIONAL,PHASE2,Number of Circulating Prostate Cells (CPCs) in...,Prostatic Neoplasms
5,5,TRB Chemedica International SA,INDUSTRY,SPONSOR,Efficacy and Safety of BUFY01 Versus SVS20 in ...,Efficacy and Safety of BUFY01 Versus SVS20 in ...,NOT_YET_RECRUITING,2024-03,ADULT OLDER_ADULT,Dry Eye Disease,TREATMENT,"BUFY01 eye drops in single-dose containers, SV...","1 to 2 drops in each eye, as often as needed, ...",INTERVENTIONAL,NaN,Signs,Dry Eye Syndromes Keratoconjunctivitis Sicca E...
6,6,Mansoura University,OTHER,PRINCIPAL_INVESTIGATOR,Caudal Dexmedetomidine Analgesia in Pediatrics .,The Analgesic Effectiveness and Safety of Upgr...,COMPLETED,2019-01-01,CHILD,164 Boys for Hypospadias Surgery Under General...,SUPPORTIVE_CARE,Caudal dexamedatomidine analgesia,Technique performance (Caudal block): classica...,INTERVENTIONAL,NaN,Time to (1st analgesic request objective pain ...,Hypospadias
7,7,Institute of Hematology & Blood Diseases Hospi...,OTHER,SPONSOR,A Study of PEG-rhG-CSF and rhG-CSF Used for Ap...,"A Randomized,Open-label Dose-discovery Study o...",RECRUITING,2022-06-05,ADULT OLDER_ADULT,Severe Aplastic Anemia,TREATMENT,PEG-rhG-CSF,"PEG-rhG-CSF group A(q7d):6mg d1,8,subcutaneous...",INTERVENTIONAL,NaN,total number of effective (ANC>0.5×109/L) days,"Anemia Anemia, Aplastic"
8,8,National Institutes of Health Clinical Center ...,NIH,SPONSOR,Screening Behavior in Adults With Hereditary H...,Illness Perceptions and the Health Belief Mode...,COMPLETED,2008-05-21,ADULT OLDER_ADULT,"Osler-Rendu-Weber Disease, Osler-Rendu Disease...",Unknown,Unknown,Unknown,OBSERVATIONAL,Unknown,Unknown,"Telangiectasis Telangiectasia, Hereditary Hemo..."
9,9,"Cascade Pharmaceuticals, Inc",OTHER,SPONSOR,"A Study to Assess the Safety, Tolerability, Ph...","A Phase II, Randomize

## Key Insights: Categorical Variable Detection

Identify likely categorical columns using a heuristic approach:
- **Dynamic Threshold**: Uses the maximum of an absolute cutoff (20) and a percentage-based threshold (1% of total rows)
- This adaptive approach works better for datasets of varying sizes
- Columns with unique values below the threshold are likely categorical and candidates for separate lookup tables

**Note**: This helped identify categorical vs continuous variables during initial exploration.
```

In [55]:
print("_" * 60)
print("Key Insights:")

absolute_cutoff = 20   # FIXME: I don't like this being hardcoded, but works for now
percent_cutoff = 0.01  # 1% of rows, this is probably too high for large datasets
n_rows = df.shape[0]
dynamic_cutoff = max(absolute_cutoff, int(n_rows * percent_cutoff))

for col in df.columns:
    unique_vals = df[col].nunique(dropna=True)
    if unique_vals <= dynamic_cutoff:
        uniques_sample = df[col].dropna().unique()[:50]
        print(f"Column: {col}")
        print(f"  - Number of uniques: {unique_vals}")
        print(f"  - Sample uniques: {list(uniques_sample)}\n")


____________________________________________________________
Key Insights:
Column: Organization Class
  - Number of uniques: 9
  - Sample uniques: ['OTHER', 'INDUSTRY', 'NIH', 'OTHER_GOV', 'NETWORK', 'FED', 'INDIV', 'UNKNOWN', 'Unknown']

Column: Responsible Party
  - Number of uniques: 4
  - Sample uniques: ['SPONSOR', 'Unknown', 'PRINCIPAL_INVESTIGATOR', 'SPONSOR_INVESTIGATOR']

Column: Overall Status
  - Number of uniques: 14
  - Sample uniques: ['COMPLETED', 'TERMINATED', 'NOT_YET_RECRUITING', 'RECRUITING', 'ACTIVE_NOT_RECRUITING', 'UNKNOWN', 'WITHDRAWN', 'SUSPENDED', 'ENROLLING_BY_INVITATION', 'APPROVED_FOR_MARKETING', 'WITHHELD', 'NO_LONGER_AVAILABLE', 'AVAILABLE', 'TEMPORARILY_NOT_AVAILABLE']

Column: Standard Age
  - Number of uniques: 6
  - Sample uniques: ['ADULT OLDER_ADULT', 'CHILD', 'ADULT', 'CHILD ADULT', 'CHILD ADULT OLDER_ADULT', 'OLDER_ADULT']

Column: Primary Purpose
  - Number of uniques: 11
  - Sample uniques: ['TREATMENT', 'Unknown', 'SUPPORTIVE_CARE', 'BASIC_SCIEN